# Sentiment Analysis Hugging Face BERT Neural Network

# Step 1 . Install dependencies

In [2]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [3]:
!pip install transformers requests beutifulsoup4 numpy pandas

ERROR: Could not find a version that satisfies the requirement beutifulsoup4 (from versions: none)
ERROR: No matching distribution found for beutifulsoup4

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


# Step 2. Import libraries

In [4]:
from transformers import AutoTokenizer , AutoModelForSequenceClassification
import torch 
import requests 
from bs4 import BeautifulSoup
import re

# Step 3. Intializing Model from Hugging Face

In [5]:
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

C:\Users\Addy\AppData\Local\Programs\Python\Python37\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Addy\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [6]:
tokenizer

BertTokenizerFast(name_or_path='nlptown/bert-base-multilingual-uncased-sentiment', vocab_size=105879, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

In [7]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(105879, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

# Step 4 . Encode and Sentiment a Phrase

In [24]:
token = tokenizer.encode("Its fine but can be better!" , return_tensors="pt")

In [25]:
token[0]

tensor([  101, 10491, 12922, 10502, 10743, 10346, 16197,   106,   102])

In [26]:
tokenizer.decode(token[0])

'[CLS] its fine but can be better! [SEP]'

In [27]:
results = model (token)

In [28]:
results

SequenceClassifierOutput(loss=None, logits=tensor([[-1.9786,  0.0348,  1.7986,  1.1080, -0.8710]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [29]:
results.logits

tensor([[-1.9786,  0.0348,  1.7986,  1.1080, -0.8710]],
       grad_fn=<AddmmBackward0>)

In [30]:
int(torch.argmax(results.logits)) + 1

3

# Sentiments through Web Scraping 

# Step 1 . Send Request to Web URL and Strip the Html Content

In [98]:


url = "https://www.yellowpages.com/baltimore-md/mip/vinnys-cafe-14279791"

response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

reviews = soup.find_all("div", class_="review-response")
review_texts = [review.find("p").text.strip() for review in reviews]

reviews

[<div class="review-response"><p>Went there for dinner. Three of us total. Nobody was in the dining room (5:00 PM) Couldn't order anything fried because of a party food being cooked for the Amazon place across the street. We were fine with that. Understood that happens. The "pizza bread" that is served with/before the meal came out as we were almost done eating. Had to flag down the bartender three times to get our waitress. Again, only ones in the restaurant. My food came out cold. Sent it back still was cold. We are usually Squire's people but we thought we would try Vinny's. I have been there two other times. No  problems as I just mentioned. But last night, was just disappointing. No reason why we had to flag the bartender down to get our waitress.</p></div>,
 <div class="review-response"><p>high prices-food is average-small portions service is okay</p></div>,
 <div class="review-response"><p>Offering the best of both worlds with a sub-shop/carry out/ informal front to a sit/down e

# Step 2 . Collect the Reveiws and Pass it to Dataframe

In [100]:
import pandas as pd

df = pd.DataFrame({"Review": review_texts})



,Review
0,Went there for dinner. Three of us total. Nobo...
1,high prices-food is average-small portions ser...
2,Offering the best of both worlds with a sub-sh...
3,This is off the beaten path when you're lookin...
4,What a great little Italian rest. tucked away ...


# Step 3. Feed it to the model

In [110]:
def sentiments_score(review):
    token = tokenizer.encode(review , return_tensors="pt")
    results = model (token)
    return int(torch.argmax(results.logits)) + 1
    

In [111]:
df["Review"].iloc[1]

'high prices-food is average-small portions service is okay'

In [112]:
sentiments_score(df["Review"].iloc[1])

3

In [113]:
df["sentiment"] = df["Review"].apply(lambda x: sentiments_score(x[:512]))

# Sentiments Results from Web Scraping

In [114]:
df

,Review,sentiment
0,Went there for dinner. Three of us total. Nobo...,3
1,high prices-food is average-small portions ser...,3
2,Offering the best of both worlds with a sub-sh...,5
3,This is off the beaten path when you're lookin...,2
4,What a great little Italian rest. tucked away ...,5
